<a href="https://colab.research.google.com/github/samhzc/Summer-2025-ECE-597-Group10/blob/main/SPAM_HAM_Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import kagglehub
import pandas as pd
import numpy as np
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [23]:
# Dataset can be found here: https://www.kaggle.com/datasets/venky73/spam-mails-dataset

from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/Collab_Files/SPAM/spam_ham_dataset.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Data Preprocessing

In [26]:
df['text'] = df['text'].str.lower()

# Clean URLs special characters
def clean_text(text):
    text = re.sub(r'\S+@\S+', ' ', text)  # remove email addresses
    text = re.sub(r'http\S+|www\S+', ' ', text)  # remove URLs
    text = re.sub(r'\d+', ' ', text)  # remove numbers
    text = re.sub(r'[^\w\s]', ' ', text)  # remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # remove extra whitespace
    text = re.sub(r'subject', '', text).strip()# remove subject
    return text

df['text'] = df['text'].apply(clean_text)

# Stop words + Lemmatization = Tokenize
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def tokenize_and_lemmatize(text):
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

df['text'] = df['text'].apply(tokenize_and_lemmatize)

### Train Test Split


In [48]:
X = df['text']
y = df['label_num']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y  # Keeps class distribution balanced
)

### Naive Bayes

In [49]:
# Get vocabulary, vectorize text
V = set()
for i in range(len(X_train)):
    V.update(X_train.iloc[i].split())

vocab_list = sorted(V)
word2idx = {word: idx for idx, word in enumerate(vocab_list)}

# N-samples by V matrix
vectorized_X_train = np.zeros((len(X_train), len(V)))

for i, text in enumerate(X_train):
    for word in text.split():
        vectorized_X_train[i, word2idx[word]] += 1

vectorized_X_test = np.zeros((len(X_test), len(vocab_list)))

for i, text in enumerate(X_test):
    for word in text.split():
        if word in word2idx:
            vectorized_X_test[i, word2idx[word]] += 1

In [50]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(vectorized_X_train, y_train)
print("Accuracy:", model.score(vectorized_X_test, y_test))

Accuracy: 0.9729468599033816
